In [6]:
# https://medium.com/@nikasa1889/a-guide-to-receptive-field-arithmetic-for-convolutional-neural-networks-e0f514068807#1780
import math
def receptive_field(n, layers):
    """
    Parameters
    ----------
    n: int, input size
    layers: array of triplets: (k,s,p)
        k: kernel size
        s: stride
        p: padding
    
    Returns
    -------
    n: output size
    j: jump size
    r: size of receptive field
    start: center position of the receptive field of the first output feature
    """
    r = 1
    j = 1
    start = 0.5
    for k, s, p in layers:
        n = math.floor( (n - k + p)/s ) + 1
        r = r + (k-1)*j
        start = start + ( (k-1)/2 - p)*j
        j = j*s

    return int(n), j, r, start

In [50]:
# Example:
layers = [
    (3, 1, 1) # kernel_size, stride, padding
#     (32, 4, 28),
#     (1, 2, 0) # , # maxpool
 
]
p = receptive_field(320, layers)
print("Output size = {:.1f}     Stride = {:2d}    RF size = {:2d}    Start = {}".format( p[0], p[1], p[2], p[3]))

Output size = 320.0     Stride =  1    RF size =  3    Start = -0.5


In [116]:
#Compute input size that leads to a 1x1 output size, among other things   


def outFromIn(isz, layernum = 9, net= None ):
    if layernum>len(net): layernum=len(net)

    totstride = 1
    insize = isz
    #for layerparams in net:
    for layer in range(layernum):
        filters, fsize, stride, pad = net[layer]
        outsize = (insize - fsize + 2*pad) / stride + 1
        insize = outsize
        totstride = totstride * stride
    return outsize, totstride

def inFromOut( layernum = 9, net = None ):
    if layernum>len(net): 
        layernum=len(net)
    outsize = 1
    #for layerparams in net:
    for layer in reversed(range(layernum)):
        filters, fsize, stride, pad = net[layer]

        outsize = ((outsize -1)* stride) + fsize
#         print('\t\t\t\t Layer : ', layer ,'filters: ', filters, ' kernel size: ', fsize, 'stride: ',stride, 'padding: ',pad, 'outsize:',outsize)
    RFsize = outsize
    return RFsize
 


In [183]:
# [filter size, stride, padding]

# convnet =[[11,4,0],[3,2,0],[5,1,2],[3,2,0],[3,1,1],[3,1,1],[3,1,1],[3,2,0],[6,1,0]]
# layer_name = ['conv1','pool1','conv2','pool2','conv3','conv4','conv5','pool5','fc6-conv']
# imsize = 227

convnet =[ [64, 3,1,1],  [64, 3,1,1],  [64, 2,2,0],
          [128, 3,1,1], [128, 3,1,1], [128, 2,2,0],
          [256, 3,1,1], [256, 3,1,1], [256, 3,1,1], [256, 2,2,0],
          [512, 3,1,1], [512, 3,1,1], [512, 3,1,1], [512, 2,2,0],
          [512, 3,1,1], [512, 3,1,1], [512, 3,1,1], [512, 2,2,0],
         [4096,7,1,3]]  
layer_name = ['conv1_1', 'conv1_2', 'pool1',
              'conv2_1', 'conv2_2', 'pool2',
              'conv3_1', 'conv3_2', 'conv3_3', 'pool3',
              'conv4_1', 'conv4_2', 'conv4_3', 'pool4',
              'conv5_1', 'conv5_2', 'conv5_3', 'pool5',
              'fc6']
imsize = 1024
in_channels = 3

print("layer output sizes given image = %dx%d" % (imsize, imsize))
print("")
print(" Layer   Name     Channels  KernelSz  Stride Padding  OutputSize Stride RF Size")
print(" -----   -------- --------  --------  ------ -------  ---------- ------ -------")

for i in range(len(convnet)):
    p = outFromIn(imsize,i+1, convnet)
    rf = inFromOut(i+1, convnet)
    l  = convnet[i]
    print(" {:3d} \t {:7s}  {:6d}     ({:2d},{:2d})  {:4d}  {:5d}  {:10d}  {:6d}  {:6d}".
          format(i,layer_name[i],l[0],l[1],l[1],l[2],l[3],int(p[0]), p[1], rf))
    
#     print("")
#     print("Layer {} : {:5s} ".format(i,layer_name[i]))
#     print("\t Channels: {:2d} \t Kernel sz:({:2d},{:2d}) \t Stride = {:2d}   \t Padding = {:2d}".
#           format(l[0],l[1],l[1],l[2],l[3])) 
#     print("\t\t\t Output size = {:.1f} \t Stride = {:2d} \t RF size = {:2d}".format( p[0], p[1], rf))
#     weights = l[1]**2 * 64 * in_channels
#     in_channels = l[0]
#     print("\t\t\t Shared weights: {} \t Bias weights: {} \t Total weights: {}".format(weights ,  l[0], weights+l[0] ))

layer output sizes given image = 1024x1024

 Layer   Name     Channels  KernelSz  Stride Padding  OutputSize Stride RF Size
 -----   -------- --------  --------  ------ -------  ---------- ------ -------
   0 	 conv1_1      64     ( 3, 3)     1      1        1024       1       3
   1 	 conv1_2      64     ( 3, 3)     1      1        1024       1       5
   2 	 pool1        64     ( 2, 2)     2      0         512       2       6
   3 	 conv2_1     128     ( 3, 3)     1      1         512       2      10
   4 	 conv2_2     128     ( 3, 3)     1      1         512       2      14
   5 	 pool2       128     ( 2, 2)     2      0         256       4      16
   6 	 conv3_1     256     ( 3, 3)     1      1         256       4      24
   7 	 conv3_2     256     ( 3, 3)     1      1         256       4      32
   8 	 conv3_3     256     ( 3, 3)     1      1         256       4      40
   9 	 pool3       256     ( 2, 2)     2      0         128       8      44
  10 	 conv4_1     512     ( 3, 3)  

In [180]:
# [filter size, stride, padding]
#Assume the two dimensions are the same
#Each kernel requires the following parameters:
# - k_i: kernel size
# - s_i: stride
# - p_i: padding (if padding is uneven, right padding will higher than left padding; "SAME" option in tensorflow)
# 
#Each layer i requires the following parameters to be fully represented: 
# - n_i: number of feature (data layer has n_1 = imagesize )
# - j_i: distance (projected to image pixel distance) between center of two adjacent features
# - r_i: receptive field of a feature in layer i
# - start_i: position of the first feature's receptive field in layer i (idx start from 0, negative means the center fall into padding)


convnet =[ [64, 3,1,1],  [64, 3,1,1],  [64, 2,2,0],
          [128, 3,1,1], [128, 3,1,1], [128, 2,2,0],
          [256, 3,1,1], [256, 3,1,1], [256, 3,1,1], [256, 2,2,0],
          [512, 3,1,1], [512, 3,1,1], [512, 3,1,1], [512, 2,2,0],
          [512, 3,1,1], [512, 3,1,1], [512, 3,1,1], [512, 2,2,0],
         [4096,7,1,3]]  
layer_names = ['conv1_1', 'conv1_2', 'pool1',
              'conv2_1', 'conv2_2', 'pool2',
              'conv3_1', 'conv3_2', 'conv3_3', 'pool3',
              'conv4_1', 'conv4_2', 'conv4_3', 'pool4',
              'conv5_1', 'conv5_2', 'conv5_3', 'pool5',
              'fc6']
imsize = 128
in_channels = 3
 
# convnet =   [[11,4,0],[3,2,0],[5,1,2],[3,2,0],[3,1,1],[3,1,1],[3,1,1],[3,2,0],[6,1,0], [1, 1, 0]]
# layer_names = ['conv1','pool1','conv2','pool2','conv3','conv4','conv5','pool5','fc6-conv', 'fc7-conv']
# imsize = 227

def outFromIn2(conv, layerIn):
    '''
    layer_in[0] : number of feature (data layer has n_1 = imagesize )
    layer_in[1] : distance (projected to image pixel distance) between center of two adjacent features
    layer_in[2] : receptive field of a feature in layer i
    layer_in[3] : position of the first feature's receptive field in layer i 
                  (idx start from 0, negative means the center fall into padding)

    conv[0] : k - filter size
    conv[1] : s - stride
    conv[2] : p - padding 
    '''    
    n_in = layerIn[0]
    j_in = layerIn[1]
    r_in = layerIn[2]
    start_in = layerIn[3]
    k = conv[0]
    s = conv[1]
    p = conv[2]

    n_out = math.floor((n_in - k + 2*p)/s) + 1
    actualP = (n_out-1)*s - n_in + k 
    pR = math.ceil(actualP/2)
    pL = math.floor(actualP/2)

    j_out = j_in * s
    r_out = r_in + (k - 1)*j_in
    start_out = start_in + ((k-1)/2 - pL)*j_in
    return n_out, j_out, r_out, start_out
  
def printLayer(layer, layer_name):
    print(layer_name + ":")
    print("\t n features: %s \n \t jump: %s \n \t receptive size: %s \t start: %s " % (layer[0], layer[1], layer[2], layer[3]))

    
layerInfos = []
# if __name__ == '__main__':
#first layer is the data layer (image) with n_0 = image size; j_0 = 1; r_0 = 1; and start_0 = 0.5
print ("-------Net summary------")
currentLayer = [imsize, 1, 1, 0.5]
printLayer(currentLayer, "input image")

for i in range(len(convnet)):
    currentLayer = outFromIn2(convnet[i], currentLayer)
    layerInfos.append(currentLayer)
    printLayer(currentLayer, layer_names[i])

print ("------------------------")
# layer_name = raw_input ("Layer name where the feature in: ")
layer_idx = layer_names.index(layer_names)
# idx_x = int(raw_input ("index of the feature in x dimension (from 0)"))
# idx_y = int(raw_input ("index of the feature in y dimension (from 0)"))

n = layerInfos[layer_idx][0]
j = layerInfos[layer_idx][1]
r = layerInfos[layer_idx][2]
start = layerInfos[layer_idx][3]
assert(idx_x < n)
assert(idx_y < n)

print ("receptive field: (%s, %s)" % (r, r))
print ("center: (%s, %s)" % (start+idx_x*j, start+idx_y*j))

-------Net summary------
input image:
	 n features: 128 
 	 jump: 1 
 	 receptive size: 1 	 start: 0.5 
conv1_1:
	 n features: 23 
 	 jump: 3 
 	 receptive size: 64 	 start: 31.0 
conv1_2:
	 n features: -12 
 	 jump: 9 
 	 receptive size: 253 	 start: 122.5 
pool1:
	 n features: -35 
 	 jump: 18 
 	 receptive size: 820 	 start: 388.0 
conv2_1:
	 n features: -53 
 	 jump: 54 
 	 receptive size: 3106 	 start: 1531.0 
conv2_2:
	 n features: -59 
 	 jump: 162 
 	 receptive size: 9964 	 start: 4960.0 
pool2:
	 n features: -91 
 	 jump: 324 
 	 receptive size: 30538 	 start: 15085.0 
conv3_1:
	 n features: -114 
 	 jump: 972 
 	 receptive size: 113158 	 start: 56071.0 
conv3_2:
	 n features: -122 
 	 jump: 2916 
 	 receptive size: 361018 	 start: 180001.0 
conv3_3:
	 n features: -125 
 	 jump: 8748 
 	 receptive size: 1104598 	 start: 551791.0 
pool3:
	 n features: -188 
 	 jump: 17496 
 	 receptive size: 3335338 	 start: 1658413.0 
conv4_1:
	 n features: -232 
 	 jump: 52488 
 	 receptive s

ValueError: ['conv1_1', 'conv1_2', 'pool1', 'conv2_1', 'conv2_2', 'pool2', 'conv3_1', 'conv3_2', 'conv3_3', 'pool3', 'conv4_1', 'conv4_2', 'conv4_3', 'pool4', 'conv5_1', 'conv5_2', 'conv5_3', 'pool5', 'fc6'] is not in list